In [ ]:
'''
1. 使用 tf.data.TextLineDataset 加载文本数据
在 TensorFlow 之中加载文本数据最常用的方式就是采用 TensorFlow 中的内置函数使用 tf.data.TextLineDataset 加载文本数据进行加载。

由于该 API 的存在，在 TensorFlow 之中加载数据变得非常简单、快捷。

在这里，我们先使用谷歌仓库中的 txt 作为一个示例，大家可以使用自己的 txt 文件进行测试。
'''

import tensorflow as tf 
import os

txt_path=tf.keras.utils.get_file("derby.txt",origin="https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt")

# 使用 tf.data.TextLineDataset 函数来加载 txt 文件，该函数会将其自动转化为 tf.data.Dataset 对象
# 然后我们对每条数据进行了映射处理，因为数据集需要含有标签，而我们的 txt 不含标签
# 因此我们使用 0 作为暂时的标签；
dataset=tf.data.TextLineDataset(txt_path).map(lambda x:(x,0))

# 再者我们使用 shuffle 对数据集进行了随机化处理，然后又进行了分批的处理
dataset.shuffle(1000).batch(32)

print(dataset)
# 查看了前四条数据
for data in labeled_dataset.take(4):
    print(data)

In [ ]:
'''
2. 使用编码将数据进行编码
我们可以使用 tensorflow_dataset.features.text.Tokenizer 对象进行编码处理
该对象能够将接收到的句子进行编码。
同时，我们可以通过 tensorflow_dataset.features.text.TokenTextEncoder 函数进行编码器的构建。
'''
import tensorflow_datasets as tfds

# 进行编码处理
tokenizer = tfds.features.text.Tokenizer()

vocab = set()
for text, l in dataset:
  token = tokenizer.tokenize(text.numpy())
  vocab.update(token)

print(len(vocab))

# 然后我们可以进行编码操作（以下映射方式参考于 TensorFlow 官方文档）：
# 定义编码器
encoder = tfds.features.text.TokenTextEncoder(vocab)

def encode(text, label):
  encoded_text = encoder.encode(text.numpy())
  return encoded_text, label

# 使用tf.py_function进行映射
# 这是因为，如果在 map 函数之中调用 Tensor.numpy() 函数会报错，
# 因此需要使用 tf.py_function 进行映射操作；
def encode_map_fn(text, label):
  encoded_text, label = tf.py_function(encode, inp=[text, label], Tout=(tf.int32, tf.int32))

  # 手动设置形状Shape
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label

# 进行编码处理
encoded_data_set = dataset.map(encode_map_fn)
print(encoded_data_set)
for data in encoded_data_set.take(4):
  print(data)

